In [2]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [3]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=round(np.float64(float(energy_string)),5)
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [16]:
#def init_hyperparameters():
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/mii/Desktop/SiMnFe/train/'

test_path='/home/mii/Desktop/SiMnFe/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
#series_num=3 #input of D
global extend_num, move_num
extend_num=1000
move_num=get_energy(train_path+'00000/')

    
E_Si=-0.038385
E_Mn=-1.475739
E_Fe=-1.264795

base_pxrd_s=mg.Structure.from_file('/home/mii/Desktop/SiMnFe/train/00000/CONTCAR')
base_pxrd=patt_xrd.get_pattern(base_pxrd_s)
#init_hyperparameters()

In [17]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##input_data_to_model
###
def get_xrdmat3(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    xrd_data4.y=xrd_data4.y-base_pxrd.y
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if abs(xrd_data4.y[i])>0.00000000001:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.dot(100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4
###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###
def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
##input_data_for_G
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat3(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [18]:
class GCNNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GCNNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

In [28]:
G1= GCNNet()
G1.load_state_dict(torch.load('/home/mii/Desktop/SiMnFe/G1_GCN-SFM-520.pkl'))

<All keys matched successfully>

In [29]:
Etotal_GCN_test=[]
Etotal_DFT_test=[]
Ebinding_GCN_test=[]
Ebinding_DFT_test=[]
MAEset_total=[]
MSEset_total=[]
MAEset_binding=[]
MSEset_binding=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
  

In [30]:
start=time.time()
#err0set=[]
      
for path_ in testfile:

    try:
        #DFT_energy
        etotal=get_energy(path_)        
        Etotal_DFT_test.append(etotal)
        
        ebinding_DFT=(etotal-(31*E_Si+1*E_Mn+2*E_Fe))/34      
        Ebinding_DFT_test.append(ebinding_DFT)
    except:
        print("errorDFT",path_)
        
        
    try:
        #GCN_energy
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        GCN_total=Gout.data.numpy().mean()
        GCN_total=inverse_transform(GCN_total)
        
        
        Etotal_GCN_test.append(GCN_total)
        
        Ebinding_GCN=(GCN_total-(31*E_Si+1*E_Mn+2*E_Fe))/34 
        Ebinding_GCN_test.append(Ebinding_GCN)
    
        
        mae_total=abs(GCN_total-etotal)
        mse_total=(GCN_total-etotal)**2
        MAEset_total.append(mae_total)
        MSEset_total.append(mse_total)
        
        mae_binding=abs(Ebinding_GCN-ebinding_DFT)
        mse_binding=(Ebinding_GCN-ebinding_DFT)**2
        MAEset_binding.append(mae_binding)
        MSEset_binding.append(mse_binding)
    except:
        print("errorG",path_)
    
end=time.time()
print(end-start)



152.31833815574646


In [14]:
def cal_mean(Alist):
    A=np.asarray(Alist)
    mean_val=A.mean()
    print(mean_val)

In [31]:
cal_mean(MAEset_total)
cal_mean(MSEset_total)
cal_mean(MAEset_binding)
cal_mean(MSEset_binding)

0.2041206971984711
0.06398101331261648
0.006003549917602079
5.534689732925286e-05


In [32]:
ttEtotal_GCN=[]
ttEtotal_DFT=[]
ttEbinding_GCN=[]
ttEbinding_DFT=[]
ttMAEset_total=[]
ttMSEset_total=[]
ttMAEset_binding=[]
ttMSEset_binding=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)


In [33]:
start=time.time()
#err0set=[]
      
for path_ in testfile:

    try:
        #DFT_energy
        etotal=get_energy(path_)        
        Etotal_DFT_test.append(etotal)
        
        ebinding_DFT=(etotal-(31*E_Si+1*E_Mn+2*E_Fe))/34        
        ttEbinding_DFT.append(ebinding_DFT)
    except:
        print("errorDFT",path_)
        
        
    try:
        #GCN_energy
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        GCN_total=Gout.data.numpy().mean()
        GCN_total=inverse_transform(GCN_total)
        
        
        ttEtotal_GCN.append(GCN_total)
        
        Ebinding_GCN=(GCN_total-(31*E_Si+1*E_Mn+2*E_Fe))/34
        ttEbinding_GCN.append(Ebinding_GCN)
    
        
        mae_total=abs(GCN_total-etotal)
        mse_total=(GCN_total-etotal)**2
        ttMAEset_total.append(mae_total)
        ttMSEset_total.append(mse_total)
        
        mae_binding=abs(Ebinding_GCN-ebinding_DFT)
        mse_binding=(Ebinding_GCN-ebinding_DFT)**2
        ttMAEset_binding.append(mae_binding)
        ttMSEset_binding.append(mse_binding)
    except:
        print("errorG",path_)
    
end=time.time()
print(end-start)



152.78944849967957


In [34]:
cal_mean(ttMAEset_total)
cal_mean(ttMSEset_total)
cal_mean(ttMAEset_binding)
cal_mean(ttMSEset_binding)

0.21111365763878706
0.06829422849660226
0.006209225224670176
5.907805233270072e-05
